In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from tqdm import tqdm
import torch

In [5]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')

Maybe helpful if you use CUDA 11.3
```shell
pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
```
More versions: https://pytorch.org/get-started/previous-versions/

In [6]:
device = 'cpu'
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
model = model.to(device)

cpu


Change the source videos here:
- tom_scott_videos_all.csv
- two_minute_papers_all.csv

In [7]:
videos = pd.read_csv('../data/subtitles/tom_scott_videos_all_subtitles.csv')
videos

,id,title,subtitles
0,0,Keeping the world's longest railroad tunnel safe,The Gotthard Base Tunnel is the longest and de...
1,1,Why do YouTubers clap at the start of videos?,To make it easy to sychronise the audio and vi...
2,2,This massive truck makes artificial earthquakes,"You know in the first Jurassic Park movie, the..."
3,3,How the US Postal Service reads terrible handw...,Almost all the letters that get sent in the Un...
4,4,"This clock was famous, but the internet ruined...","Normally, when I’m filming somewhere like this..."
...,...,...,...
317,317,The Second Largest Freezer in Norway,TOM SCOTT: The Youth Winter Olympic Games have...
318,318,Why Wuppertal's Suspended Monorail Wasn't The ...,"There's a reason that the old, retro-futuristi..."
319,319,The Russian Woodpecker of Chernobyl: How To Se...,"In the 1970s, during the Cold War, if you want..."
320,320,Radioactive Bananas in Chernobyl,"I'm in the abandoned city of Pripyat, in Ukrai..."


In [8]:
summaries = list()
for _, row in tqdm(videos.iterrows(), total=videos.shape[0]):
    batch = tokenizer(row['subtitles'], truncation=True, max_length=1024, return_tensors='pt')
    generated_ids = model.generate(batch["input_ids"].to(device), min_length=256, max_length=512)
    summary = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    summaries.append({'title': row['title'],  'summary': summary[0]})

  1%|          | 3/322 [04:27<7:53:48, 89.12s/it]


KeyboardInterrupt: 

In [ ]:
summaries[0]

In [23]:
df = pd.DataFrame(summaries)
df.to_csv('title_summary.csv', index=True, index_label='id', header=True)

In [ ]:
test = pd.read_csv('title_summary.csv')
test